In [ ]:
from tensorflow import keras

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [ ]:
print(y_train_full.mean())
print(y_train_full.max())

In [ ]:
def build_model(norm_layer):
  input = keras.layers.Input(shape=[8])y_ta
  norm_data = norm_layer(input)
  hidden1 = keras.layers.Dense(200, activation="relu")(norm_data)
  out_put = keras.layers.Dense(1)(hidden1)
  model = keras.Model(inputs=[input], outputs=[out_put])

  model.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

  return model

In [ ]:
norm_layer = keras.layers.Normalization()

model = build_model(norm_layer)
check_point = keras.callbacks.ModelCheckpoint("check_point_house.h5", save_weights_only=True, save_best_only=True)
early_stop = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
norm_layer.adapt(X_train)
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[check_point, early_stop])

In [ ]:
model = build_model(norm_layer)
model_1 = model.load_weights("/content/check_point_house.h5")

In [ ]:
def build_model2(norm_layer):
  input = keras.layers.Input(shape=[8])
  norm_data = norm_layer(input)
  hidden1 = keras.layers.Dense(300, activation="relu")(norm_data)
  hidden2 = keras.layers.Dense(200, activation="relu")(hidden1)
  out_put = keras.layers.Dense(1)(hidden2)
  model = keras.Model(inputs=[input], outputs=[out_put])

  model.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

  return model

In [ ]:
norm_layer = keras.layers.Normalization()

model = build_model2(norm_layer)
check_point = keras.callbacks.ModelCheckpoint("check_point_house.h5", save_weights_only=True, save_best_only=True)
early_stop = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[check_point, early_stop])

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot()

In [ ]:
def build_model3(norm_layer):
  input = keras.layers.Input(shape=[8])
  norm_data = norm_layer(input)
  hidden1 = keras.layers.Dense(300, activation="relu", kernel_regularizer= keras.regularizers.l2(0.01))(norm_data)
  hidden2 = keras.layers.Dense(200, activation="relu", kernel_regularizer= keras.regularizers.l2(0.01))(hidden1)
  concat = keras.layers.Concatenate()([norm_data, hidden2])
  model = keras.Model(inputs=[input], outputs=[concat])

  model.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

  return model

In [ ]:
norm_layer = keras.layers.Normalization()

model = build_model3(norm_layer)
check_point = keras.callbacks.ModelCheckpoint("check_point_house.h5", save_weights_only=True, save_best_only=True)
early_stop = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[check_point, early_stop])

##### Keras Subclassing

In [ ]:
class MyModel(keras.Model):
  def __init__(self, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.norm_layer = keras.layers.Normalization()
    self.hidden1 = keras.layers.Dense(300, activation=activation, kernel_regularizer= keras.regularizers.l2(0.01))
    self.hidden2 = keras.layers.Dense(200, activation=activation, kernel_regularizer= keras.regularizers.l2(0.01))
    self.concat = keras.layers.Concatenate()
    self.out_put = keras.layers.Dense(1)

  def adapt_norm_layer(self, input):
    self.norm_layer.adapt(input)

  def call(self, input):
    self.adapt_layer(input)
    input_ = self.norm_layer(input)
    hidden1_ = self.hidden1(input_)
    hidden2_ = self.hidden2(hidden1_)
    concat = self.concat([input_, hidden2_])
    return self.out_put(concat)

In [ ]:
def build_model4():
  model_dynamic = MyModel(name="Mapsa_Model")

  model_dynamic.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

  return model_dynamic

In [ ]:
model = build_model4()
check_point = keras.callbacks.ModelCheckpoint("check_point_house.h5", save_weights_only=True, save_best_only=True)
early_stop = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[check_point, early_stop])

In [ ]:
model.summary()

In [ ]:
from pathlib import Path
from time import strftime
import os

def get_run_logdir(root_logdir="my_logs"):
  return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

In [ ]:
%pip install -q -U tensorboard-plugin-profile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 71.3 MB/s eta 0:00:00


In [ ]:
model = build_model4()
check_point = keras.callbacks.ModelCheckpoint("check_point_house.h5", save_weights_only=True, save_best_only=True)
early_stop = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[check_point, early_stop, tensorboard_cb])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

In [ ]:
%pip install keras-tuner

In [ ]:
import keras_tuner as kt

def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value=0, max_value=8,default=2)
  n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
  learning_rate = hp.Float("learning_rate", min_value=1e-4,max_value=1e-2, sampling="log")
  optimizer = hp.Choice("optimizer", values=["sgd", "adam"])


  if optimizer == "sgd":
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  else:
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

  model = keras.Sequential()
  model.add(keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,activation="relu"))
  model.add(keras.layers.Dense(10, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
  return model

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

In [ ]:
random_search_tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=5, overwrite=True, directory="my_fashion_mnist", project_name="my_rnd_search", seed=42)
random_search_tuner.search(X_train_full, y_train_full, epochs=10,validation_data=(X_test, y_test))